In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def get_pages(page):
    reqUrl = f"https://github.com/topics?page={page}"
    return reqUrl

In [ ]:
def get_page_content(page_url):
    response = requests.get(page_url)
    if response.status_code != 200:
        raise Exception(f'Failed to load page {page_url}')
    page_doc = BeautifulSoup(response.text, 'html.parser')
    return page_doc

In [ ]:
def get_topic_title(doc):
    topic_title_tag = doc.find_all('p', {'class':'f3 lh-condensed mb-0 mt-1 Link--primary'})
    topic_title = []
    for item in topic_title_tag:
        topic_title.append(item.text.strip())
    return topic_title
   
def get_topic_desc(doc):
    topic_desc_tag = doc.find_all('p', {'class':'f5 color-fg-muted mb-0 mt-1'})
    topic_desc = []
    for item in topic_desc_tag:
        topic_desc.append(item.text.strip())
    return topic_desc

def get_topic_url(doc):
    topic_url_tag = doc.find_all('a', {'class':'no-underline flex-1 d-flex flex-column'})
    base_url = 'https://github.com'
    topic_url = []
    for item in topic_url_tag:
        topic_url.append(base_url + item['href'])
    return topic_url

def scrape_topics_info(page_doc):
    topics_dict = {
        'topic_title': get_topic_title(page_doc),
        'topic_desc': get_topic_desc(page_doc),
        'topic_url': get_topic_url(page_doc)
    }
    return pd.DataFrame(topics_dict)

def import_to_csv(dataFrame):
    dataFrame.to_csv('./topics/topics.csv', index = None)
    
def update_and_filter_data(new_df):
    re_index = [*range(100, len(new_df))]
    update_df = new_df.drop(re_index)
    import_to_csv(update_df)

In [ ]:
def access_topic_page_url(df, topic_num):
    return df['topic_url'][topic_num]

def parse_star_count(stars_str):
    stars_str = stars_str.strip()
    if stars_str[-1] == 'k':
        return int(float(stars_str[:-1])*1000)
    else:
        return int(stars_str)

def get_repo_info(repo_tag, star_tag):
    base_url = 'https://github.com'
    a_tag = repo_tag.find_all('a')
    username = a_tag[0].text.strip()
    repo_name = a_tag[1].text.strip()
    stars = parse_star_count(star_tag.text.strip())
    repo_url = base_url + a_tag[1]['href']
    return username, repo_name, stars, repo_url
    
def get_topic_repos(topic_doc):
    repo_tag = topic_doc.find_all('h3', {'class':'f3 color-fg-muted text-normal lh-condensed'})
    star_tag = topic_doc.find_all('span', {'class':'Counter js-social-count'})
    
    topic_repos_dict = {
        'username': [], 
        'repo_name': [], 
        'stars': [],
        'repo_url': []
        }
    
    for i in range(len(repo_tag)):
        repo_info = get_repo_info(repo_tag[i], star_tag[i])
        topic_repos_dict['username'].append(repo_info[0])
        topic_repos_dict['repo_name'].append(repo_info[1])
        topic_repos_dict['stars'].append(repo_info[2])
        topic_repos_dict['repo_url'].append(repo_info[3])
    return pd.DataFrame(topic_repos_dict)

def import_topic_repos_to_csv(df, topic):
    df.to_csv(f'./topics/repos/{topic}.csv', index=None)

In [ ]:
temp_df = pd.DataFrame()
for page in range(1, 6):
    if len(temp_df) < 100:
        url = get_pages(page)
        page_doc = get_page_content(url)
        df = scrape_topics_info(page_doc)
        temp_df = temp_df.append(df)
    else:
        break
import_to_csv(temp_df)
new_df = pd.read_csv('./topics/topics.csv')
update_and_filter_data(new_df)

In [ ]:
new_df = pd.read_csv('./topics/topics.csv')
for page in range(len(new_df)):
        url = access_topic_page_url(new_df, page)
        doc = get_page_content(url)
        df = get_topic_repos(doc)
        import_topic_repos_to_csv(df, new_df['topic_title'][page])